# **人工智能实验报告**



## **一、实验题目**

编写一个程序，实现一阶逻辑归结算法，并利用该算法求解三个逻辑推理问题

## **二、实验内容**

### **1.算法原理**

本程序主要分为两个步骤，归结与合一。具体来说是在给定的$KB$集合中，首先通过合一的替换将互为正负的文字统一，随后利用归结原理不断将两个句子生成新句子，最终推导出空集

#### **1.1归结**

归结的核心是将$KB$中的命题转换为子句形式，通过不断选取两个句子(遍历所有)，找出互补的文字，并利用归结原理将两个子句生成新子句并加入到$KB$中(注意需要利用合一算法对各个项进行必要的替换)，最终能推导出空子句则说明$KB$存在矛盾，继而由反证法说明目标命题的正确性。

归结的实现代码保存为`FolResolution.py`，需要导入合一算法`MGU.py`中的`MGU`与` term_substitution`函数。

文件中定义了的函数及作用如下：

- `is_false`：判断给定字符串表示的文字是否为负文字
- `use_substitution`：对文字进行替换，需要调用`term_substitution`
- `str_MGU`：对可以进行归结的文字进行合一，需要调用`MGU`函数
- `solve_clause`：对每一对文字判断是否互补，若是，则经过合一后进行归结，得到新句子
- `solve_prop`：归结证明的入口函数，遍历$KB$的子句编号，对现有的子句调用`solve_clause`，得到新子句后加入$KB$，直到推出空子句或无法进行推导
- `print_ans`：输出函数

#### **1.2合一**

合一的核心是对于两个互补的文字，判断他们是否可以通过替换统计，进而可以送入归结进行推理。使用最一般合一的方法，找到一个最一般的替换使得经过替换的两个文字完全相同，并在后续推理中应用该替换。其中合一先将子句解析，递归比较谓词，参数情况，以及替换，同时需要进行标记防止出现嵌套替换。

合一的实现代码保存为`MGU.py`

文件中定义的函数及作用如下：

- `formular_solve`：解析逻辑公式，将公式分解为谓词以及参数形式，该函数能处理参数中含有函数的形式($f(x)$)，但本次实验的标准输入参数均为变量或常量
- `term_substitution`：递归对给定的变量或公式应用替换，对于变量检查是否在替换表`substitution`中，对于公式则将每个参数进行替换，递归是为了保证替换内部也完成替换。
- `check`：检查变量`var`是否在项`term`中出现，防止出现变量在自身替换中替换以及嵌套替换等不合法行为
- `is_variabl`：判断字符串是否为变量，根据输入形式给出固定的变量集合`list = {'a','b','x','xx','y','yy','z','zz','u','uu','w','v'}`
- `change_var`：处理`var`与其他项的合一，在`var`是变量的情况下，如果`var`已有替换，则递归进行替换统一，`x`也同理，如果都没有则进行检查后将`var`替换为`x`并扩展替换表`substitution`
- `change`：合一的函数，接受两个需要替换的项以及替换表`substitution`。首先应用已有替换表进行替换更新，随后进行解析项的情况并分类处理，最终递归使得两个公式统一
- `MGU`：合一过程的入口函数，首先解析公式，检谓词与参数，再利用`change`对每个项进行替换，失败则返回`None`，其余返回替换表`substitution`

### **2.关键代码展示**

#### **2.1归结关键代码**

In [1]:
# 两两归结
def solve_clause(clause1, clause2, num1, num2):
    clause1 = list(clause1)
    clause2 = list(clause2)
    ans = []
    for i, list1 in enumerate(clause1):
        for j, list2 in enumerate(clause2):
            if is_false(list1) != is_false(list2):
                substitution = str_MGU(list1, list2)
                if substitution is not None:
                    new_clause = set(clause1[:i] + clause1[i+1:] + clause2[:j] + clause2[j+1:])
                    new_substitution = {use_substitution(k, substitution) for k in new_clause}
                    label = f"R[{num1}{chr(ord('a')+i)}, {num2}{chr(ord('a')+j)}] {substitution}"
                    ans.append((label, frozenset(new_substitution)))
    return ans

# 归结入口函数
def solve_prop(KB):
    clauses= []
    messages = {}
    num = 1
    flag_set = set()
    for i in KB:
        if isinstance(i, (tuple, list)):
            clause = frozenset(i)
        else:
            clause = frozenset((i,))
        clauses.append((num, clause)) # clause
        messages[num] = (clause, "1") # clause and output
        flag_set.add(clause) # flag
        num += 1
    while True:
        n = len(clauses)
        new_clauses = []
        for i in range(n):
            for j in range(i+1, n):
                num1, clause1 = clauses[i]
                num2, clause2 = clauses[j]
                ans = solve_clause(clause1, clause2, num1, num2)
                for (output, new_cal) in ans:
                    if new_cal not in flag_set:
                        messages[num] = (new_cal, output)
                        if len(new_cal) == 0:
                            print_ans(messages)
                            return messages
                        new_clauses.append((num, new_cal))
                        flag_set.add(new_cal)
                        num += 1
        if not new_clauses:
            print("无法退出新句子")
            print(messages)
            return messages
        clauses.extend(new_clauses)

#### **2.2合一关键代码**

In [2]:
# 递归替换
def term_substitution(term, substitution):
    name, args = formular_solve(term)
    # 递归必须
    if not args:
        if term in substitution:
            return term_substitution(substitution[term], substitution) # 
        else:
            return term
    
    new_args = []
    for i in args:
        new_args.append(term_substitution(i, substitution))
    return f"{name}({','.join(new_args)})"


# 替换项
def change_var(var, x, substitution):
    if not is_variable(var):
        if var == x:
            return substitution
        else:
            return None

    if var in substitution:
        return change(substitution[var], x, substitution)
    elif x in substitution:
        return change(var, substitution[x], substitution)
    else:
        if check(var, x, substitution):
            return None
        substitution[var] = x
        return substitution

# 合一
def change(t1, t2, substitution):
    t1_change = term_substitution(t1, substitution)
    t2_change = term_substitution(t2, substitution)

    if t1_change == t2_change:
        return substitution
    
    name1, args1 = formular_solve(t1_change)
    name2, args2 = formular_solve(t2_change)

    if args1 == [] and args2 == []:
        if t1_change != t2_change:
            if is_variable(t1_change):
                return change_var(t1_change, t2_change, substitution)
            elif is_variable(t2_change):
                return change_var(t2_change, t1_change, substitution)
            else:
                return None
        else:
            return substitution
    
    if args1 == []:
        if is_variable(t1_change):
            return change_var(t1_change, t2_change, substitution)
        else:
            return None
    
    if args2 == []:
        if is_variable(t2_change):
            return change_var(t2_change, t1_change, substitution)
        else:
            return None
    
    if name1 == name2 and len(args1) == len(args2):
        for i1, i2 in zip(args1, args2):
            substitution = change(i1, i2, substitution)
            if substitution is None:
                return None
    else:
        return None
    
    return substitution



def MGU(term1, term2):
    name1, args1 = formular_solve(term1)
    name2, args2 = formular_solve(term2)

    if args1 == [] and args2 == []:
        return change(term1, term2, {})

    if name1 != name2:
        return None

    substitution = {}

    if len(args1) == len(args2):
        for i1, i2 in zip(args1, args2):
            substitution = change(i1, i2, substitution)
            if substitution is None:
                return None
    else:
        return None
    
    return substitution

## **三、实验结果分析**

### **1.实验结果展示示例**

测试文件保存在`main.py`中，导入`FolResolution.py`中的`solve_prop`函数即可。首先，由于$KB$库的存储方式为集合`set`，并且为了方便使用集合判断是否重复，输出的结果中，子句的顺序可能与原本输入的不同，但是结果是正确的。
同时由于传入的$KB$库是集合`set`导致其的不确定性，从而使每次的归结推理的过程步数都不一定相同，但是对于问题2，3，推理步数基本维持在$200$和$70$左右

In [3]:
# 导入需要的函数
from FolResolution import solve_prop

# 测试三个问题
if __name__ == '__main__':
    """
    输出的归结过程中的初始KB的子句的每一文字可能和输入的顺序不一样
    同时输出归结过程的子句顺序也和初始的不一样
    这是因为使用了set存放子句集以及子句的每一个文字用于判断是否出现重复
    从归结的结果来看，归结推理的过程是正确的
    """
    KB1 = {
        ("GradStudent(sue)"),
        ("~GradStudent(x)", "Student(x)"),
        ("~Student(x)", "HardWorker(x)"),
        ("~HardWorker(sue)")
    }
    result1 = solve_prop(KB1)
    print("\n")
    print("--------------------")
    print("\n")
    KB2 = {
        ("A(tony)"),
        ("A(mike)"),
        ("A(john)"),
        ("L(tony, rain)"),
        ("L(tony, snow)"),
        ("~A(x)", "S(x)", "C(x)"),
        ("~C(y)", "~L(y, rain)"),
        ("L(z, snow)", "~S(z)"),
        ("~L(tony, u)", "~L(mike, u)"),
        ("L(tony, v)", "L(mike, v)"),
        ("~A(w)", "~C(w)", "S(w)")
    }
    result2 = solve_prop(KB2)
    print("\n")
    print("--------------------")
    print("\n")
    KB3 = {
        ("On(tony, mike)"),
        ("On(mike, john)"),
        ("Green(tony)"),
        ("~Green(john)"),
        ("~On(xx, yy)", "~Green(xx)", "Green(yy)")
    }
    result3 = solve_prop(KB3)

归结过程：
1 (~GradStudent(x), Student(x))
2 (~HardWorker(sue))
3 (~Student(x), HardWorker(x))
4 (GradStudent(sue))
5: R[1b, 3a] {} => (HardWorker(x), ~GradStudent(x))
6: R[1a, 4a] {'x': 'sue'} => (Student(sue))
7: R[2a, 3b] {'x': 'sue'} => (~Student(sue))
8: R[1b, 7a] {'x': 'sue'} => (~GradStudent(sue))
9: R[3a, 6a] {'x': 'sue'} => (HardWorker(sue))
10: R[6a, 7a] {} => []


--------------------


归结过程：
1 (~S(z), L(z, snow))
2 (L(tony, snow))
3 (A(tony))
4 (~C(w), S(w), ~A(w))
5 (~L(mike, u), ~L(tony, u))
6 (L(tony, rain))
7 (L(mike, v), L(tony, v))
8 (A(mike))
9 (~L(y, rain), ~C(y))
10 (A(john))
11 (C(x), ~A(x), S(x))
12: R[1a, 4b] {'z': 'w'} => (~C(w), L(w,snow), ~A(w))
13: R[1b, 5a] {'z': 'mike', 'u': 'snow'} => (~S(mike), ~L(tony,snow))
14: R[1b, 5b] {'z': 'tony', 'u': 'snow'} => (~L(mike,snow), ~S(tony))
15: R[1a, 11c] {'z': 'x'} => (C(x), ~A(x), L(x,snow))
16: R[2a, 5b] {'u': 'snow'} => (~L(mike,snow))
17: R[3a, 4c] {'w': 'tony'} => (S(tony), ~C(tony))
18: R[3a, 11b] {'x': 'tony'} => 

### **2.评测指标展示及分析**

由于本实验的实现归结采取的是遍历所反复遍历所有的子句以以及新生成的子句集，导致最终遍历的测试较多，但是需要存储的空间较少，使用时间换空间，对于小规模的推理，推理的时间较快且没有明显的差距，对于更大规模的推理，可采用优化中的方法实现

## **四、优化算法**

### **1.优化内容与算法**

#### **1.1优化内容**

1. 一开始采取的是遍历所有的组合的算法，该算法需要遍历所有的子句进行两两归结，从而产生大量的没有必要的推理，从而导致推理次数增加，在题目2中推理次数多达$200$次。为了优化算法减少推理次数，我们针对一阶逻辑的归结进行研究。当某个子句只有一个文字时(即单文字子句)，他与其他子句的归结起到了剪枝的效果，并且可以减少归结组合数量，同时单文字归结可以使部分句子提前化简或是推导出空句子。对于一般的两个子句归结，假设文字数分别为：$m$，$n$，则正常的遍历次数最大为$m \times n$。而单文字的归结只需要进行$m$或$n$次，次数有明显的减少。

2. 同时算法增加一个过滤操作：对于新推出的句子，如果该子句$a$包含了已有的子句$b$，则可以认为该$a$对归结推理没有帮助，故可以不加入子句库$KB$，同时如果已有的子句$b$包含了新推出的子句$a$，则认为已有的子句是多余的，可以去除。通过过滤的操作可以大量减少$KB$中无用的子句，从而减少归结推理的步骤

#### **1.2算法实现**

1. 修改`solve_clause`函数，增加两个分支判断。对于传入的子句`clause1`和`clause2`，分别判断处理二者为单文字的情况，进而减少遍历两个子句所有文字的组合次数。
2. 修改`solve_prop`函数，在每一轮归结之前，按照所有子句数量进行排序，从而优先处理单文字的子句，提升剪枝的效果
3. 修改`solve_prop`函数，增加一个过滤操作，对于新推出的子句，判断其包含关系，去除包含的子句

### **优化部分代码**

In [4]:
# 两两归结代码--优化
def solve_clause(clause1, clause2, num1, num2):
    clause1_list = list(clause1)
    clause2_list = list(clause2)
    ans = []
    # 优化处：新增两个分支判断，处理单文字情况
    if len(clause1_list) == 1:
        clause = clause1_list[0]
        for i, list2 in enumerate(clause2_list):
            if is_false(clause) != is_false(list2):
                substitution = str_MGU(clause, list2)
                if substitution is not None:
                    new_clause = set(clause2_list) - {list2}
                    new_clause = {use_substitution(l, substitution) for l in new_clause}
                    label = f"R[{num1}a, {num2}{chr(ord('a')+i)}] {substitution}"
                    ans.append((label, frozenset(new_clause)))
    elif len(clause2_list) == 1:
        clause = clause2_list[0]
        for i, list1 in enumerate(clause1):
            if is_false(clause) != is_false(list1):
                substitution = str_MGU(list1, clause)
                if substitution is not None:
                    new_clause = set(clause1_list) - {list1}
                    new_clause = {use_substitution(l, substitution) for l in new_clause}
                    label = f"R[{num1}{chr(ord('a')+i)}, {num2}a] {substitution}"
                    ans.append((label, frozenset(new_clause)))
    else:
        for i, list1 in enumerate(clause1_list):
            for j, list2 in enumerate(clause2_list):
                if is_false(list1) != is_false(list2):
                    substitution = str_MGU(list1, list2)
                    if substitution is not None:
                        new_clause = set(clause1_list[:i] + clause1_list[i+1:] + clause2_list[:j] + clause2_list[j+1:])
                        new_substitution = {use_substitution(k, substitution) for k in new_clause}
                        label = f"R[{num1}{chr(ord('a')+i)}, {num2}{chr(ord('a')+j)}] {substitution}"
                        ans.append((label, frozenset(new_substitution)))
    return ans
# 总实现函数--优化
def solve_prop(KB):
    clauses= []
    messages = {}
    num = 1
    flag_set = set()
    for i in KB:
        clause = frozenset(i)
        clauses.append((num, clause)) # clause
        messages[num] = (clause, "1") # clause and output
        flag_set.add(clause) # flag
        num += 1
    while True:
        clauses.sort(key=lambda item: len(item[1]))
        n = len(clauses)
        new_clauses = []
        for i in range(n):
            for j in range(i+1, n):
                num1, clause1 = clauses[i]
                num2, clause2 = clauses[j]
                ans = solve_clause(clause1, clause2, num1, num2)
                for (output, new_cal) in ans:
                    if new_cal not in flag_set:
                        new_clauses.append((output, new_cal))
        if not new_clauses:
            print("无法退出新句子")
            print(messages)
            return messages
        # new fliter_function
        filtered_clause = []
        for output, clause in new_clauses:
            # new_clause include i
            flag = False
            for i in flag_set:
                if issub(i, clause):
                    flag = True
                    break
            if flag == True:
                continue
            # i include new_clause
            remove_set = set()
            for i in flag_set:
                if issub(clause, i):
                    remove_set.add(i)
            if remove_set:
                flag_set = flag_set - remove_set
                clauses = [(n_val, cl) for n_val, cl in clauses if cl not in remove_set]

            if clause not in flag_set:
                filtered_clause.append((output, clause))
                messages[num] = (clause, output)
                clauses.append((num, clause))
                flag_set.add(clause)
                num += 1
        for output, clause in filtered_clause:
            if len(clause) == 0:
                print_ans(messages)
                return messages

### **3.优化实验结果展示实例**

优化算法的代码保存为`FolResolution2.py`，测试方式同上

In [5]:
from FolResolution2 import solve_prop

if __name__ == '__main__':
    """
    输出的归结过程中的初始KB的子句的每一文字可能和输入的顺序不一样
    这是因为使用了set存放子句的每一个文字判断是否出现重复
    但是从归结的结果来看，归结推理的过程是正确的
    """
    KB1 = [
        {"GradStudent(sue)"},
        {"~GradStudent(x)", "Student(x)"},
        {"~Student(x)", "HardWorker(x)"},
        {"~HardWorker(sue)"}
    ]
    
    result1 = solve_prop(KB1)

    print("\n")
    print("--------------------")
    print("\n")

    KB2 = [
        {"A(tony)"},
        {"A(mike)"},
        {"A(john)"},
        {"L(tony, rain)"},
        {"L(tony, snow)"},
        {"~A(x)", "S(x)", "C(x)"},
        {"~C(y)", "~L(y, rain)"},
        {"L(z, snow)", "~S(z)"},
        {"~L(tony, u)", "~L(mike, u)"},
        {"L(tony, v)", "L(mike, v)"},
        {"~A(w)", "~C(w)", "S(w)"}
    ]

    result2 = solve_prop(KB2)

    print("\n")
    print("--------------------")
    print("\n")

    KB3 = [
        {"On(tony, mike)"},
        {"On(mike, john)"},
        {"Green(tony)"},
        {"~Green(john)"},
        {"~On(xx, yy)", "~Green(xx)", "Green(yy)"}
    ]
    result3 = solve_prop(KB3)

归结过程：
1 (GradStudent(sue))
2 (~GradStudent(x), Student(x))
3 (~Student(x), HardWorker(x))
4 (~HardWorker(sue))
5: R[1a, 2a] {'x': 'sue'} => (Student(sue))
6: R[4a, 3b] {'x': 'sue'} => (~Student(sue))
7: R[2b, 3a] {} => (HardWorker(x), ~GradStudent(x))
8: R[1a, 7b] {'x': 'sue'} => (HardWorker(sue))
9: R[4a, 7a] {'x': 'sue'} => (~GradStudent(sue))
10: R[5a, 6a] {} => []


--------------------


归结过程：
1 (A(tony))
2 (A(mike))
3 (A(john))
4 (L(tony, rain))
5 (L(tony, snow))
6 (C(x), ~A(x), S(x))
7 (~L(y, rain), ~C(y))
8 (~S(z), L(z, snow))
9 (~L(mike, u), ~L(tony, u))
10 (L(mike, v), L(tony, v))
11 (~C(w), S(w), ~A(w))
12: R[1a, 6b] {'x': 'tony'} => (S(tony), C(tony))
13: R[1a, 11c] {'w': 'tony'} => (S(tony), ~C(tony))
14: R[2a, 6b] {'x': 'mike'} => (C(mike), S(mike))
15: R[2a, 11c] {'w': 'mike'} => (~C(mike), S(mike))
16: R[3a, 6b] {'x': 'john'} => (C(john), S(john))
17: R[3a, 11c] {'w': 'john'} => (S(john), ~C(john))
18: R[4a, 7a] {'y': 'tony'} => (~C(tony))
19: R[4a, 9b] {'u': 'rain'} =>

### **4.优化结果分析**

上述输出即为优化结果，对于规模较小的推理，如$KB1$的推理，单文字策略以及过滤并没有改善，但是对于大规模的问题，如$KB2$的推理，应用单文字策略有明显的优化效果，从原来的$182$次到$55$次，同理对于$KB3$，从原来的$73$到$27$，减少了大量的推理次数。